# Feature Extraction

In [ ]:
# Dinucleotide-based Cross Covariance (DCC)

import os
bil='python iLearn-nucleotide-acc.py --file examples/train.txt --method DCC --type DNA --lag 5 --format csv --out DCC.csv'
os.system(bil)

In [ ]:
# Pseudo k-tupler Composition (PseKNC) 

import os
bil='python iLearn-nucleotide-Pse.py --file examples/train.txt --method PseKNC --type DNA --lamada 2  --weight 0.1 --format csv --out PseKNC.csv'
os.system(bil)

In [ ]:
# Series Correlation Pseudo Dinucleotide Composition (SCPseDNC)

import os
bil='python iLearn-nucleotide-Pse.py --file examples/train.txt --method SCPseDNC --type DNA --lamada 2  --weight 0.1 --format csv --out SCPseDNC.csv'
os.system(bil)

In [ ]:
# Parallel Correlation Pseudo Dinucleotide Composition (PCPseDNC)
import os
bil='python iLearn-nucleotide-Pse.py --file examples/train.txt --method PCPseDNC --type DNA --lamada 2  --weight 0.1 --format csv --out PCPseDNC.csv'
os.system(bil)

In [ ]:
# Kmer
import os
bil='python iLearn-nucleotide-basic.py --file examples/train.txt --method Kmer --format csv --out Kmer.csv'
os.system(bil)

In [ ]:
# Multisource biological features (NPCP)

In [ ]:
import sys, os,re, platform
import numpy as np
import pandas as pd
import itertools
from sklearn.preprocessing import LabelEncoder
import argparse
from sklearn.ensemble import AdaBoostClassifier
 
from Bio.Seq import Seq

In [ ]:
DNAelements = 'ACGT'


elements = DNAelements

m2 = list(itertools.product(elements, repeat=2))
m3 = list(itertools.product(elements, repeat=3))
m4 = list(itertools.product(elements, repeat=4))
m5 = list(itertools.product(elements, repeat=5))

def readFASTAs(filename):
    with open(filename, 'r') as file:
        v = []
        genome = ''
        for line in file:
            if line[0] != '>':
                genome += line.strip()
            else:
                v.append(genome.upper())
                genome = ''
        v.append(genome.upper())
        del v[0]
        return v
filename = readFASTAs("/Path/")

In [ ]:


DNAelements = 'ACGT'


elements = DNAelements

m2 = list(itertools.product(elements, repeat=2))
m3 = list(itertools.product(elements, repeat=3))
m4 = list(itertools.product(elements, repeat=4))
m5 = list(itertools.product(elements, repeat=5))

def readFASTAs(filename):
    with open(filename, 'r') as file:
        v = []
        genome = ''
        for line in file:
            if line[0] != '>':
                genome += line.strip()
            else:
                v.append(genome.upper())
                genome = ''
        v.append(genome.upper())
        del v[0]
        return v

def saveCSV(name, X, Y, type):
    if type == 'test':
        F = open((name + '_testDataset.csv'), 'w')

    else:
        if type == 'optimum':
            F = open((name + '_optimumDataset.csv'), 'w')

    for x, y in zip(X, Y):
        for each in x:
            F.write(str(each) + ',')
        F.write(str(int(y)) + '\n')
    F.close()

def readLabels(fileName):
    with open(fileName, 'r') as file:
        v = []
        for line in file:
            if line != '\n':
                v.append((line.replace('\n', '')).replace(' ', ''))
        return v

def kmers(seq, k):
    v = []
    for i in range(len(seq) - k + 1):
        v.append(seq[i:i + k])
    return v

def pseudoKNC(x, k):
    t = []
    for i in range(1, k + 1, 1):
        v = list(itertools.product(elements, repeat=i))
        for i in v:
            t.append(x.count(''.join(i)))
    return t

def zCurve(x):
    t = []
    T = x.count('T'); A = x.count('A'); C = x.count('C'); G = x.count('G');
    x_ = (A + G) - (C + T)
    y_ = (A + C) - (G + T)
    z_ = (A + T) - (C + G)
    t.append(x_); t.append(y_); t.append(z_)
    return t

def gcContent(x):
    t = []
    T = x.count('T')
    A = x.count('A');
    C = x.count('C');
    G = x.count('G');
    t.append( (G + C) / (A + C + G + T)  * 100.0 )
    return t


def cumulativeSkew(x):
    
    t = []
    T = x.count('T')
    A = x.count('A');
    C = x.count('C');
    G = x.count('G');

    GCSkew = (G-C)/(G+C)
    ATSkew = (A-T)/(A+T)

    t.append(GCSkew)
    t.append(ATSkew)
    return t


def atgcRatio(x):
    t = []
    T = x.count('T')
    A = x.count('A');
    C = x.count('C');
    G = x.count('G');

    t.append( (A+T)/(G+C) )
    return t


def monoMonoKGap(x, g):  # k=1, m=1
    m = m2
    t = []
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 2)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[-1] == gGap[1]:
                    C += 1
            t.append(C)
    return t

def monoDiKGap(x, g):  # k=1, m=2
    t = []
    m = m3
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 3)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[-2] == gGap[1] and v[-1] == gGap[2]:
                    C += 1
            t.append(C)
    return t

def diMonoKGap(x, g):  # k=2, m=1
    t = []
    m = m3
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 3)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[1] == gGap[1] and v[-1] == gGap[2]:
                    C += 1
            t.append(C)
    return t
def monoTriKGap(x, g):  # k=1, m=3
    t = []
    m = m4
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 4)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[-3] == gGap[1] and v[-2] == gGap[2] and v[-1] == gGap[3]:
                    C += 1
            t.append(C)
    return t
def triMonoKGap(x, g):  # k=3, m=1
    t = []
    m = m4
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 4)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[1] == gGap[1] and v[2] == gGap[2] and v[-1] == gGap[3]:
                    C += 1
            t.append(C)
    return t
def diDiKGap(x, g): # k=2, m=2
    m = m4
    t = []
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 4)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[1] == gGap[1] and v[-2] == gGap[2] and v[-1] == gGap[3]:
                    C += 1
            t.append(C)
    return t
def diTriKGap(x, g):  # k=2, m=3
    m = m5
    t = []
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 5)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[1] == gGap[1] and v[-3] == gGap[2] and v[-2] == gGap[3] and v[-1] == gGap[4]:
                    C += 1
            t.append(C)
    return t
def triDiKGap(x, g):  # k=3, m=2
    m = m5
    t = []
    for i in range(1, g + 1, 1):
        V = kmers(x, i + 5)
        for gGap in m:
            C = 0
            for v in V:
                if v[0] == gGap[0] and v[1] == gGap[1] and v[2] == gGap[2] and v[-2] == gGap[3] and v[-1] == gGap[4]:
                    C += 1
            t.append(C)
    return t
def generateFeatures(kGap, kTuple, x, y):
        feature = []
        t = zCurve(x)         
        for item in t:
            feature.append(item)
        t= gcContent(x)           
        for item in t:
            feature.append(item)
        t = cumulativeSkew(x)      
        for item in t:
            feature.append(item)
        t = atgcRatio(x)         
        for item in t:
            feature.append(item)
        t = pseudoKNC(x, kTuple)            
        for item in t:
            feature.append(item)
        t = monoMonoKGap(x, kGap)      
        for item in t:
            feature.append(item)
        t = monoDiKGap(x, kGap)        
        for item in t:
            feature.append(item)
        t = monoTriKGap(x, kGap)       
        for item in t:
            feature.append(item)
        t = diMonoKGap(x, kGap)        
        for item in t:
            feature.append(item)
        t = diDiKGap(x, kGap)          
        for item in t:
            feature.append(item)
        t = diTriKGap(x, kGap)        
        for item in t:
            feature.append(item)
        t = triMonoKGap(x, kGap)       
        for item in t:
            feature.append(item)
        t = triDiKGap(x, kGap)         
        for item in t:
            feature.append(item)
        feature.append(y)
        return feature



In [ ]:
# Generate Train data Features
X = readFASTAs("/Path/")
print('loading sequence complete')
Y = readLabels("/Path/")
print('loading label complete')
Y = LabelEncoder().fit_transform(Y)
assert len(X)==len(Y)
T = []
print('start generating features')
for x, y in zip(X, Y):
    
    feature = generateFeatures(5, 3, x, y)
    print(len(feature))
    T.append(feature)
    print(len(T))
T = np.array(T)

X = T[:,:-1]
Y = T[:,-1]
print('Finish generating features')


In [ ]:
# X = pd.DataFrame(X)
# X.to_csv("/home/bilal/Biomed/Dataset/ilearn/kGAP_kTuple_train.csv")

In [ ]:
np.savetxt("/path/name.csv", X,
              delimiter = ",")

In [ ]:
# Generate Independent data features
X = readFASTAs("/path/")
print('loading sequence complete')
Y = readLabels("/path/")
print('loading label complete')
Y = LabelEncoder().fit_transform(Y)
assert len(X)==len(Y)
T = []
print('start generating features')
for x, y in zip(X, Y):
    
    feature = generateFeatures(5, 3, x, y)
    print(len(feature))
    T.append(feature)
    print(len(T))
T = np.array(T)

X = T[:,:-1]
Y = T[:,-1]
print('Finish generating features')


In [ ]:
# X = pd.DataFrame(X)
# X.to_csv("/home/bilal/Biomed/Dataset/ilearn/kGAP_kTuple_ind.csv")

In [ ]:
np.savetxt("/path/name.csv", X,
              delimiter = ",")

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

# Load the data
data = pd.read_csv("/data/", header = None, names = ['sequence'])

# Clean the data
data = data.dropna()

# Extract features
vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='char')
X = vectorizer.fit_transform(data['sequence']).toarray()

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)
